In [1]:
from abc import (
    ABC,
    abstractmethod,
    abstractclassmethod,
    abstractproperty
)
from dataclasses import dataclass
from typing import Generator, Iterator, Optional
from IPython.display import display, HTML

import pygraphviz as pgv
import base64
import sympy as sm
from sympy.abc import a, b, c, d, e, f
from sympy.core.expr import Expr
from sympy.core.symbol import Symbol
from sympy.logic.boolalg import Boolean, BooleanTrue, BooleanFalse

In [2]:
sm.init_session()

IPython console for SymPy 1.14.0 (Python 3.12.4-64-bit) (ground types: python)

These commands were executed:
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.14.0/



# Abstract classes

In [3]:
@dataclass(frozen=True)
class Expression(ABC):

    @abstractproperty
    def name(self) -> str:
        ...

    @abstractclassmethod
    def is_final(cls) -> bool:
        ...

    @abstractmethod
    def to_sympy(self) -> Expr:
        ...

    @abstractmethod
    def __iter__(self) -> Iterator['Expression']:
        ...

    @abstractmethod
    def replace(self, old: 'Expression', new: 'Expression') -> 'Expression':
        ...


@dataclass(frozen=True)
class Operator(Expression, ABC):

    @abstractclassmethod
    def args_number(cls) -> int:
        ...

    @abstractproperty
    def args(self) -> tuple[Expression]:
        ...

    @classmethod
    def is_final(cls) -> bool:
        return False

    def __repr__(self):
        return f'{self.name}({", ".join(repr(arg) for arg in self.args)})'

    def __iter__(self):
        yield self
        for arg in self.args:
            for i in arg:
                yield i

    def replace(self, old: 'Expression', new: 'Expression') -> 'Expression':
        if self == old:
            return new
        return type(self)(*list(arg.replace(old, new) for arg in self.args))


@dataclass(frozen=True)
class Function(Operator, ABC):

    def __str__(self):
        return f'{self.name}({", ".join(
            str(arg)
            if arg.is_final() or self.args_number() == 1
            else f"({arg})"
            for arg in self.args
        )})'

    def to_sympy(self) -> Expr:
        return sm.Function(self.name)(*list(arg.to_sympy()
                                            for arg in self.args))


@dataclass(frozen=True)
class InfixOperator(Operator, ABC):

    def __str__(self):
        return f" {self.name} ".join(
            str(arg)
            if arg.is_final()
            or isinstance(arg, Function)
            or isinstance(self, Equals)
            else f"({arg})"
            for arg in self.args
        )


@dataclass(frozen=True)
class Literal(Expression):
    _name: str

    @property
    def name(self) -> str:
        return self._name

    @classmethod
    def is_final(cls) -> bool:
        return True

    def __str__(self):
        return self.name

    def __repr__(self):
        return f'Literal({repr(self.name)})'

    def to_sympy(self) -> Expr:
        return Symbol(self.name)

    def __iter__(self) -> 'Literal':
        yield self

    def replace(self, old: 'Expression', new: 'Expression') -> 'Expression':
        if self == old:
            return new
        return self


class Rule(ABC):

    def __new__(cls):
        raise Exception("Static class. Initialization is not allowed.")

    @abstractclassmethod
    def can_apply(cls, expression: Expression) -> bool:
        ...

    @abstractclassmethod
    def apply(cls, expression: Expression) -> Expression:
        ...

    @abstractclassmethod
    def get_pattern(cls) -> 'Exception':
        ...

    @classmethod
    def get_appliable(cls, expression) -> \
            tuple['Expression', 'Expression'] | tuple[None, None]:
        for expr in expression:
            if cls.can_apply(expr):
                return expr, cls.apply(expr)
        return None, None

# More classes

In [4]:
@dataclass(frozen=True)
class Addition(InfixOperator):
    _arg1: Expression
    _arg2: Expression

    @classmethod
    def args_number(cls) -> int:
        return 2

    @property
    def args(self) -> tuple[Expression]:
        return self._arg1, self._arg2

    @property
    def name(self) -> str:
        return '+'

    def to_sympy(self) -> Expr:
        return sm.Add(self._arg1.to_sympy(), self._arg2.to_sympy())


@dataclass(frozen=True)
class Multiplication(InfixOperator):
    _arg1: Expression
    _arg2: Expression

    @classmethod
    def args_number(cls) -> int:
        return 2

    @property
    def args(self) -> tuple[Expression]:
        return self._arg1, self._arg2

    @property
    def name(self) -> str:
        return '*'

    def to_sympy(self) -> Expr:
        return sm.Mul(self._arg1.to_sympy(), self._arg2.to_sympy())


@dataclass(frozen=True)
class Increment(Function):
    _arg: Expression

    @classmethod
    def args_number(cls) -> int:
        return 1

    @property
    def args(self) -> tuple[Expression]:
        return self._arg,

    @property
    def name(self) -> str:
        return 's'


@dataclass(frozen=True)
class Double(Function):
    _arg: Expression

    @classmethod
    def args_number(cls) -> int:
        return 1

    @property
    def args(self) -> tuple[Expression]:
        return self._arg,

    @property
    def name(self) -> str:
        return 'd'


@dataclass(frozen=True)
class Equals(InfixOperator):
    _arg1: Expression
    _arg2: Expression

    @classmethod
    def args_number(cls) -> int:
        return 2

    @property
    def args(self) -> tuple[Expression]:
        return self._arg1, self._arg2

    @property
    def name(self) -> str:
        return '='

    def to_sympy(self) -> Expr:
        return sm.Eq(self._arg1.to_sympy(), self._arg2.to_sympy())


_0 = Literal('0')


class RuleA(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Addition) \
            and expression.args[1] == _0

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            return expression.args[0]

    @classmethod
    def get_pattern(cls) -> Exception:
        x = Literal('x')
        return Equals(Addition(x, _0), x)


class RuleB(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Addition) \
            and isinstance(expression.args[1], Increment)

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            x = expression.args[0]
            y = expression.args[1].args[0]
            return Increment(Addition(x, y))

    @classmethod
    def get_pattern(cls) -> Expression:
        x = Literal('x')
        y = Literal('y')
        return Equals(Addition(x, Increment(y)), Increment(Addition(x, y)))


class RuleC(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Multiplication) \
            and expression.args[1] == _0

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            return _0

    @classmethod
    def get_pattern(cls) -> Expression:
        x = Literal('x')
        return Equals(Multiplication(x, _0), _0)


class RuleD(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Multiplication) \
            and isinstance(expression.args[1], Increment)

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            x = expression.args[0]
            y = expression.args[1].args[0]
            return Addition(
                Multiplication(x, y), x
            )

    @classmethod
    def get_pattern(cls) -> Expression:
        x = Literal('x')
        y = Literal('y')
        return Equals(
            Multiplication(x, Increment(y)),
            Addition(Multiplication(x, y), x)
        )


class RuleE(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Double) \
            and expression.args[0] == _0

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            return _0

    @classmethod
    def get_pattern(cls) -> Expression:
        return Equals(Double(_0), _0)


class RuleF(Rule):

    @classmethod
    def can_apply(cls, expression: Expression) -> bool:
        return isinstance(expression, Double) \
            and isinstance(expression.args[0], Increment)

    @classmethod
    def apply(cls, expression: Expression) -> Expression:
        if cls.can_apply(expression):
            x = expression.args[0].args[0]
            return Increment(Increment(Double(x)))

    @classmethod
    def get_pattern(cls) -> Expression:
        x = Literal('x')
        return Equals(Double(Increment(x)), Increment(Increment(Double(x))))

In [5]:
f1 = Multiplication(Literal('x'), Increment(_0))
print(f1)
f11 = RuleD.apply(f1)
print(f11)
print(RuleA.apply(f11))

x * s(0)
(x * 0) + x
None


In [16]:
for i in 'ABCDEF':
    rule = eval(f'Rule{i}')
    print(f'Правило {i}:', rule.get_pattern())

formula: x * s(0)
Правило A: x + 0 = x
Правило B: x + s(y) = s(x + y)
Правило C: x * 0 = 0
Правило D: x * s(y) = (x * y) + x
Правило E: d(0) = 0
Правило F: d(s(x)) = s(s(d(x)))


In [7]:
f1 = Multiplication(Literal('x'), Increment(_0))
print('formula:', f1)
m, r = RuleD.get_appliable(f1)
print(f'{m} => {r}')
f2 = f1.replace(m, r)
print(Equals(f1, f2))
m, r = RuleC.get_appliable(f2)
print(f'{m} => {r}')
f3 = f2.replace(m, r)
print(Equals(f2, f3))


formula: x * s(0)
x * s(0) => (x * 0) + x
x * s(0) = (x * 0) + x
x * 0 => 0
(x * 0) + x = 0 + x


# Доказательство формул
## Правила

In [17]:
for i in 'ABCDEF':
    rule = eval(f'Rule{i}')
    print(f'Правило {i}:', rule.get_pattern())

Правило A: x + 0 = x
Правило B: x + s(y) = s(x + y)
Правило C: x * 0 = 0
Правило D: x * s(y) = (x * y) + x
Правило E: d(0) = 0
Правило F: d(s(x)) = s(s(d(x)))


## Формула 1

In [93]:
x = Literal('x')
_1 = Increment(_0)
inc_x = Increment(x)
formula1 = Equals(Multiplication(x, _1), x)
print('Доказать:', formula1)
print()
print(formula1)
left, right = formula1.args
print(f"Применим правило D [{RuleD.get_pattern()}] к левой части, где x=x, y=0:")
step1 = left.replace(*RuleD.get_appliable(left))
print(Equals(step1, right))
print(f"Применим правило C [{RuleC.get_pattern()}] к левой части, где x=x:")
step2 = step1.replace(*RuleC.get_appliable(step1))
new_formula1 = Equals(step2, right)
print(new_formula1)
print("Получили новую формулу:", new_formula1)
print()
suggestion = new_formula1
print('Индукционное предположение P(x):', suggestion)
base = new_formula1.replace(x, _0)
print("База                       P(0):", base)
print(f"Согласно правилу A [{RuleA.get_pattern()}], где x=0, база верна.")
induction = new_formula1.replace(x, inc_x)
print()
print("Докажем индукционный переход P(x) => P(s(x)).")
print('P(s(x)):', induction)
print(f"Применим правило B [{RuleB.get_pattern()}] к левой части, где x=0, y=x:")
step3 = induction.replace(*RuleB.get_appliable(induction))
print(step3)
print("Целевое высказывание соответствует применеию "
      "s к обеим частям индукционного предположения, "
      "значит, индуктивный переход доказан.")

Доказать: x * s(0) = x

x * s(0) = x
Применим правило D [x * s(y) = (x * y) + x] к левой части, где x=x, y=0:
(x * 0) + x = x
Применим правило C [x * 0 = 0] к левой части, где x=x:
0 + x = x
Получили новую формулу: 0 + x = x

Индукционное предположение P(x): 0 + x = x
База                       P(0): 0 + 0 = 0
Согласно правилу A [x + 0 = x], где x=0, база верна.

Докажем индукционный переход P(x) => P(s(x)).
P(s(x)): 0 + s(x) = s(x)
Применим правило B [x + s(y) = s(x + y)] к левой части, где x=0, y=x:
s(0 + x) = s(x)
Целевое высказывание соответствует применеию s к обеим частям индукционного предположения, значит, индуктивный переход доказан.


## Коммутативность +
```
a + b = b + a
```
### Лемма 1
```
L1: a + 0 = 0 + a
P(x): x + 0 = 0 + x 
Применим правило A [x + 0 = x] к левой части, где x=x
P(x): x = 0 + x
P(0): 0 + 0 = 0 + 0 -- верно, так как левая и правая части равны
P(x) => P(s(x))
P(s(x)): s(x) + 0 = 0 + s(x)
Применим правило A [x + 0 = x] к левой части, где x=s(x)
P(s(x)): s(x) = 0 + s(x)
Применим правило B [x + s(y) = s(x + y)] к правой части, где x=0, y=x
P(s(x)): s(x) = s(0 + x)
Так как мы предполагаем, что x = 0 + x, то заменим в левой части x на 0 + x
P(s(x)): s(0 + x) = s(0 + x) -- верно, так как левая и правая части равны => L1 - верно
```
### Лемма 2
```
L2: s(a + b) = s(a) + b
P(x): s(a + x) = s(a) + x
P(0): s(a + 0) = s(a) + 0
Применим правило A [x + 0 = x] к левой части, где x=a
P(0): s(a) = s(a) + 0
Применим правило A [x + 0 = x] к правой части, где x=s(a)
P(0): s(a) = s(a) -- верно, так как левая и правая части равны

P(x) => P(s(x))
P(s(x)): s(a + s(x)) = s(a) + s(x)
Применим правило B [x + s(y) = s(x + y)] к левой части, где x=a, y=x
P(s(x)): s(s(a + x)) = s(a) + s(x)
Применим правило B [x + s(y) = s(x + y)] к правой части, где x=s(a), y=x
P(s(x)): s(s(a + x)) = s(s(a) + x)
Так как мы предполагаем, что s(a + x) = s(a) + x, то заменим в левой части s(a + x) на s(a) + x
P(s(x)): s(s(a) + x) = s(s(a) + x) -- верно, так как левая и правая части равны => L2 - верно
```
### Доказательство a + b = b + a
```
P(x): a + x = x + a
P(0): a + 0 = 0 + a -- верно по L1

P(x) => P(s(x))
P(s(x)): a + s(x) = s(x) + a
Применим правило B [x + s(y) = s(x + y)] к левой части, где x=a, y=x
P(s(x)): s(a + x) = s(x) + a
Так как мы предполагаем, что a + x = x + a, то заменим в левой части a + x на x + a
P(s(x)): s(x + a) = s(x) + a
Заменим в левой части s(x + a) на s(x) + a по L2
P(s(x)): s(x) + a = s(x) + a -- верно, так как левая и правая части равны => a + b = b + a
```

## Коммутативность *
```
a * b = b * a
```
### Лемма 3
```
L3: a * 0 = 0 * a
P(x): x * 0 = 0 * x 
Применим правило C [x * 0 = 0] к левой части, где x=x
P(x): 0 = 0 * x 
P(0): 0 * 0 = 0 * 0 -- верно, так как левая и правая части равны
P(x) => P(s(x))

P(s(x)): s(x) * 0 = 0 * s(x)
Применим правило C [x * 0 = 0] к левой части, где x=s(x)
P(s(x)): 0 = 0 * s(x)
Применим правило D [x * s(y) = (x * y) + x] к правой части, где x=0, y=x
P(s(x)): 0 = (0 * x) + 0
Применим правило A [x * 0 = x] к правой части, где x=(0 * x)
P(s(x)): 0 = 0 * x
Так как мы предполагаем, что 0 = 0 * x, то заменим в левой части 0 на 0 * x
P(s(x)): 0 * x = 0 * x -- верно, так как левая и правая части равны => L3 - верно
```
### Лемма 4
```
L4: a * (b + c) = a * b + a * c
P(x): a * (b + x) = a * b + a * x
P(0): a * (b + 0) = a * b + a * 0
Применим правило A [x + 0 = x] к левой части, где x=b
P(0): a * b = a * b + a * 0
Применим правило C [x * 0 = 0] к правой части, где x=a
P(0): a * b = a * b + 0
Применим правило A [x + 0 = x] к правой части, где x=(a * b)
P(0): a * b = a * b -- верно, так как левая и правая части равны

P(x) => P(s(x))
P(s(x)): a * (b + s(x)) = a * b + a * s(x)
Применим правило B [x + s(y) = s(x + y)] к левой части, где x=b, y=x
P(s(x)): a * s(b + x) = a * b + a * s(x)
Применим правило D [x * s(y) = (x * y) + x] к левой части, где x=a, y=(b + x)
P(s(x)): a * (b + x) + a = a * b + a * s(x)
Применим правило D [x * s(y) = (x * y) + x] к правой части, где x=a, y=x
P(s(x)): a * (b + x) + a = a * b + a * x + a
Так как мы предполагаем, что a * (b + x) = a * b + a * x, то заменим в левой части a * (b + x) на a * b + a * x
P(s(x)): a * b + a * x + a = a * b + a * x + a -- верно, так как левая и правая части равны => L4 - верно
```
### Лемма 5
```
L5: s(a) * b = (a * b) + b
P(x): s(a) * x = (a * x) + x
P(0): s(a) * 0 = (a * 0) + 0
Применим правило A [x + 0 = x] к правой части, где x=(a * 0)
P(0): s(a) * 0 = a * 0
Применим правило C [x * 0 = 0] к обеим частям, для левой x=s(a), для правой - x=a
P(0): 0 = 0 -- верно, так как левая и правая части равны

P(x) => P(s(x))
P(s(x)): s(a) * s(x) = (a * s(x)) + s(x)
Применим правило B [x + s(y) = s(x + y)] к правой части, где x=(a * s(x)), y=x
P(s(x)): s(a) * s(x) = s(a * s(x) + x)
Применим правило D [x * s(y) = (x * y) + x] к левой части, где x=s(a), y=x
P(s(x)): (s(a) * x) + s(a) = s(a * s(x) + x)
Применим правило B [x + s(y) = s(x + y)] к левой части, где x=(s(a) * x), y=a
P(s(x)): s(s(a) * x + a) = s(a * s(x) + x)
Применим правило D [x * s(y) = (x * y) + x] к правой части, где x=a, y=x
P(s(x)): s(s(a) * x + a) = s((a * x) + a + x)
Так как мы предполагаем, что s(a) * x = (a * x) + x, то заменим в левой части s(a) * x на (a * x) + x
P(s(x)): s((a * x) + x + a) = s((a * x) + a + x)
Заменим в левой части x + a на a + x
P(s(x)): s((a * x) + a + x) = s((a * x) + a + x) -- верно, так как левая и правая части равны => L5 - верно
```
### Доказательство a * b = b * a
```
P(x): a * x = x * a
P(0): a * 0 = 0 * a -- верно по L3

P(x) => P(s(x))
P(s(x)): a * s(x) = s(x) * a 
Применим L5 [s(a) * b = (a * b) + b] к правой части, где a=x, b=a
P(s(x)): a * s(x) = (x * a) + a
Применим правило D [x * s(y) = (x * y) + x] к левой части, где x=a, y=x
P(s(x)): (a * x) + a = (x * a) + a
Так как мы предполагаем, что a * x = x * a, то заменим в левой части a * x на x * a
P(s(x)): (x * a) + a = (x * a) + a -- верно, так как левая и правая части равны => a * b = b * a - верно
```

## Формула 2
```
Доказать: s(0) * x = x

Так как s(0) * x = x * s(0) и x * s(0) = x, то s(0) * x = x
Доказано
```
## Формула 3
```
Доказать: x + s(y) = s(x) + y
Применим правило B [x + s(y) = s(x + y)] к левой части, где x=x, y=y
s(x + y) = s(x) + y
Верно, согласно L2: [s(a + b) = s(a) + b], где a=x, b=y
Доказано
```

Доказать: s(0) * x = x

Так как s(0) * x = x * s(0) и x * s(0) = x, то s(0) * x = x